# Initialize the Database

In [ ]:
import os
import pathlib
import glob
import cv2 as cv2
import numpy as np
import xml.etree.ElementTree as ET
import pandas as pd
import tensorflow as tf
import skimage.io as io


In [ ]:
!mkdir -p /data/sets/nuimages  # Make the directory to store the nuImages dataset in.

!wget https://www.nuscenes.org/data/nuimages-v1.0-mini.tgz  # Download the nuImages mini split.

!tar -xf nuimages-v1.0-mini.tgz -C /data/sets/nuimages  # Uncompress the nuImages mini split.

!pip install nuscenes-devkit &> /dev/null  # Install nuImages.

--2023-11-07 11:55:15--  https://www.nuscenes.org/data/nuimages-v1.0-mini.tgz
Resolving www.nuscenes.org (www.nuscenes.org)... 13.227.219.45, 13.227.219.18, 13.227.219.6, ...
Connecting to www.nuscenes.org (www.nuscenes.org)|13.227.219.45|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 117929607 (112M) [application/x-tar]
Saving to: ‘nuimages-v1.0-mini.tgz’

nuimages-v1.0-mini. 100%[===================>] 112.47M  26.1MB/s    in 4.7s    

2023-11-07 11:55:20 (23.8 MB/s) - ‘nuimages-v1.0-mini.tgz’ saved [117929607/117929607]



In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
from nuimages import NuImages

nuim = NuImages(dataroot='/data/sets/nuimages', version='v1.0-mini', verbose=True, lazy=True)

Loading nuImages tables for version v1.0-mini...
Done loading in 0.001 seconds (lazy=True).


Categories that are annotated

In [ ]:
# The NuScenes class holds several tables. Each table is a list of records, and each record is a dictionary.
# For example the first record of the category table is stored at

#nusc.category[0]['name']

#these are the categories available
cat = []
for i in range(len(nuim.category)):
    print(nuim.category[i]['name'])
    cat.append(nuim.category[i]['name'])



Loaded 25 category(s) in 0.000s,
animal
flat.driveable_surface
human.pedestrian.adult
human.pedestrian.child
human.pedestrian.construction_worker
human.pedestrian.personal_mobility
human.pedestrian.police_officer
human.pedestrian.stroller
human.pedestrian.wheelchair
movable_object.barrier
movable_object.debris
movable_object.pushable_pullable
movable_object.trafficcone
static_object.bicycle_rack
vehicle.bicycle
vehicle.bus.bendy
vehicle.bus.rigid
vehicle.car
vehicle.construction
vehicle.ego
vehicle.emergency.ambulance
vehicle.emergency.police
vehicle.motorcycle
vehicle.trailer
vehicle.truck


# classes that we are detecting :

We merge adult, child, police officer, construction worker into a single class called pedestrian
We are detecting:
- pedestrian
- car
- bicycle

In [ ]:
classes = ['human.pedestrian.adult', 'human.pedestrian.child','human.pedestrian.police_officer','human.pedestrian.construction_worker','human.pedestrian.personal_mobility','human.pedestrian.stroller','human.pedestrian.wheelchair','vehicle.car','vehicle.bicycle']
pedestrians = ['human.pedestrian.adult', 'human.pedestrian.child','human.pedestrian.police_officer','human.pedestrian.construction_worker','human.pedestrian.personal_mobility','human.pedestrian.stroller','human.pedestrian.wheelchair']

In [ ]:
print('Total number of samples')
print(len(nuim.sample))

total_no_of_samples = len(nuim.sample)

#print('Total number of images')
#print(len(nusc.sample*6)) #6 different cameras

Total number of samples
Loaded 50 sample(s) in 0.000s,
50


# Functions

Defined the following function:

- get_sample_data (edit of nutonomy's original nusc.get_sample_data)
        
        input:(nusc, sample_data_token)
        output:path to the data, lists of 3d bounding boxes in the image (in camera coordinates),
        annotation token of annotations in the image, intrinsic matrix of the camera)
        

- threeD_to_2D
         
        input: (box (camera coordinates),intrinsic matrix))
        output : corners of the 2d bounding box in image plane

- all_3d_to_2d(boxes,anns,intrinsic)

        input : boxes in camera coordinates, list of annotation tokens of annotations in the image,
        intrinsic matrix
        output: x_min,x_max,y_min,y_max,width,height of the 2D boundings boxes of objects that are
        more than 40% visible in panoramic view of all cameras, also ensures that the center of the
        bounding boxes falls inside the image

- extract_bounding_box(i):
        
        input: sample number
        output: min x, max x, min y max y, width and height of bounding box in image coordinates
        2d bounding box of objects which are 40% visible in panoramic view of all cameras and center
        falls witin the image



In [ ]:
#from pyquaternion import Quaternion
#from nuscenes.utils.data_classes import Box
#from nuscenes.utils.geometry_utils import quaternion_slerp, box_in_image, BoxVisibility
import numpy as np
def get_sample_data(nuim, sample_data_token, selected_anntokens=None):
    """
    Returns the data path as well as all annotations related to that sample_data(single image).
    Note that the boxes are transformed into the current sensor's coordinate frame.
    :param sample_data_token: <str>. Sample_data token(image token).
    :param box_vis_level: <BoxVisibility>. If sample_data is an image, this sets required visibility for boxes.
    :param selected_anntokens: [<str>]. If provided only return the selected annotation.
    :return: (data_path <str>, boxes [<Box>], camera_intrinsic <np.array: 3, 3>)
    """

    # Retrieve sensor & pose records
    sd_record = nuim.get('sample_data', sample_data_token)
    cs_record = nuim.get('calibrated_sensor', sd_record['calibrated_sensor_token'])
    sensor_record = nuim.get('sensor', cs_record['sensor_token'])
    pose_record = nuim.get('ego_pose', sd_record['ego_pose_token'])

    sample_record = nuim.get('sample',sd_record['sample_token'])
    data_path = nuim.get_sample_data_path(sample_data_token)

    if sensor_record['modality'] == 'camera':
        cam_intrinsic = np.array(cs_record['camera_intrinsic'])
        imsize = (sd_record['width'], sd_record['height'])
    else:
        cam_intrinsic = None
        imsize = None

    # Retrieve all sample annotations and map to sensor coordinate system.
    if selected_anntokens is not None:
        boxes = list(map(nuim.get_box, selected_anntokens))
    else:
        boxes = nuim.get_boxes(sample_data_token)
        selected_anntokens = sample_record['anns']

    # Make list of Box objects including coord system transforms.
    box_list = []
    ann_list = []
    for box,ann in zip(boxes,selected_anntokens):

        # Move box to ego vehicle coord system
        box.translate(-np.array(pose_record['translation']))
        box.rotate(Quaternion(pose_record['rotation']).inverse)

        #  Move box to sensor coord system
        box.translate(-np.array(cs_record['translation']))
        box.rotate(Quaternion(cs_record['rotation']).inverse)

        if sensor_record['modality'] == 'camera' and not \
                box_in_image(box, cam_intrinsic, imsize, vis_level=box_vis_level):
            continue

        box_list.append(box)
        ann_list.append(ann)
    #this is for a single sample image
    return data_path, box_list, ann_list, cam_intrinsic #single image info

In [ ]:
def threeD_2_twoD(boxsy,intrinsic): #input is a single annotation box
    '''
    given annotation boxes and intrinsic camera matrix
    outputs the 2d bounding box coordinates as a list (all annotations for a particular sample image)
    '''
    corners = boxsy.corners()
    x = corners[0,:]
    y = corners[1,:]
    z = corners[2,:]
    x_y_z = np.array((x,y,z))
    orthographic = np.dot(intrinsic,x_y_z)
    perspective_x = orthographic[0]/orthographic[2]
    perspective_y = orthographic[1]/orthographic[2]
    perspective_z = orthographic[2]/orthographic[2]

    min_x = np.min(perspective_x)
    max_x = np.max(perspective_x)
    min_y = np.min(perspective_y)
    max_y = np.max(perspective_y)



    return min_x,max_x,min_y,max_y



def all_3d_to_2d(boxes,anns,intrinsic): #input 3d boxes, annotation key lists, intrinsic matrix (one image)
    x_min=[]
    x_max=[]
    y_min=[]
    y_max =[]
    width=[]
    height=[]
    objects_detected =[]
    orig_objects_detected =[]


    for j in range(len(boxes)): #iterate through boxes
        box=boxes[j]

        if box.name in classes: #if the box.name is in the classes we want to detect

            if box.name in pedestrians:
                orig_objects_detected.append("pedestrian")
            elif box.name == "vehicle.car":
                orig_objects_detected.append("car")
            else:
                orig_objects_detected.append("cyclist")
            #print(box)

            visibility = nusc.get('sample_annotation', '%s' %anns[j])['visibility_token'] #give annotation key
            visibility = int(visibility)


            if visibility > 1: #more than 40% visible in the panoramic view of the the cameras


                center = box.center #get boxe's center

                center = np.dot(intrinsic,center)
                center_point = center/(center[2]) #convert center point into image plane




                if center_point[0] <-100 or center_point[0] > 1700 or center_point[1] <-100 or center_point[1] >1000:
                    #if center of bounding box is outside of the image, do not annotate
                    pass

                else:
                    min_x, max_x, min_y, max_y = threeD_2_twoD(box,intrinsic) #converts box into image plane
                    w = max_x - min_x
                    h = max_y - min_y


                    x_min.append(min_x)
                    x_max.append(max_x)
                    y_min.append(min_y)
                    y_max.append(max_y)
                    width.append(w)
                    height.append(h)
                    if box.name in pedestrians:
                        objects_detected.append("pedestrian")
                    elif box.name == "vehicle.car":
                        objects_detected.append("car")
                    else:
                        objects_detected.append("cyclist")


            else:
                pass

    return x_min,x_max,y_min,y_max,width,height,objects_detected,orig_objects_detected #for a single image

In [ ]:
def extract_bounding_box(i,camera_name): #give a single sample number and camera name

    '''
    input sample number i, camera name
    outputs min x, max x, min y max y, width and height of bounding box in image coordinates
    2d bounding box
    options for camera name : CAM_FRONT, CAM_FRONT_RIGHT, CAM_FRONT_LEFT, CAM_BACK, CAM_BACK_RIGHT,CAM_BACK_LEFT
    '''

    nuim.sample[i] #one image

    camera_token = nuim.sample[i]['data']['%s' %camera_name] #one camera, get the camera token

    path, boxes, anns, intrinsic_matrix = get_sample_data(nuim,'%s' %camera_token) #gets data for one image

    x_min, x_max,y_min,y_max,width,height, objects_detected,orig_objects_detected = all_3d_to_2d(boxes,anns, intrinsic_matrix)

    return x_min, x_max, y_min, y_max, width, height, path, boxes,intrinsic_matrix, objects_detected,orig_objects_detected
    #info for a single image


In [ ]:
#Create target Directory if don't exist
import os.path
def create_annotation_directory(camera):
    current_dir =os.getcwd()
    #current_dir ="%s/annotation" %pwd
    dirName ="%s/annotation/%s_anno" %(current_dir,camera)
    if not os.path.exists(dirName):
        os.makedirs(dirName)
        print("Directory " , dirName ,  " Created ")
    else:
        print("Directory " , dirName ,  " already exists")

In [ ]:
from lxml import etree as ET
def write_xml_annotation(x_min,x_max,y_min,y_max,width,height,path,boxes,objects_detected): #single image info
    #detected_items =[]
    #import xml.etree.cElementTree as ET
    path_split = path.split("/")
    full_image_name = path_split[-1]
    name =full_image_name.split(".")[0]

    root = ET.Element("annotation")


    ET.SubElement(root, "folder").text = "%s" %camera
    ET.SubElement(root, "filename").text = "%s" %full_image_name
    ET.SubElement(root, "path").text = "%s" %path

    source = ET.SubElement(root, "source")
    ET.SubElement(source, "database").text = "nuTonomy-nuscenes"

    size = ET.SubElement(root, "size")
    ET.SubElement(size, "width").text="1600"
    ET.SubElement(size,"height").text="900"
    ET.SubElement(size,"depth").text="3"
    ET.SubElement(root, "segmented").text = "0"

    for j in range(len(objects_detected)): #

        flag_x = 0
        flag_y = 0

        ob= ET.SubElement(root, "object")
        ET.SubElement(ob,"name").text="%s" %objects_detected[j]
        ET.SubElement(ob,"pose").text="Unspecified"


        '''
        write out truncated boxes
        '''

        if x_min[j] < 0:
            x_minsy = 0
            flag_x =1

        else:
            x_minsy = x_min[j]

        if y_min[j] <0:
            y_minsy = 0
            flag_y =1

        else:
            y_minsy = y_min[j]

        if x_max[j] > 1600:
            x_maxsy = 1600
            flag_x = 1

        else:
            x_maxsy = x_max[j]

        if y_max[j] >900:
            y_maxsy = 900
            flag_y = 1

        else:
            y_maxsy = y_max[j]


        if flag_x == 1 or flag_y ==1:
            ET.SubElement(ob, "truncated").text="1"

        else:
            ET.SubElement(ob, "truncated").text="0"




        ET.SubElement(ob, "difficult").text="0"

        bb = ET.SubElement(ob,"bndbox")


        ET.SubElement(bb,"xmin").text="%s" %x_minsy
        ET.SubElement(bb,"ymin").text="%s" %y_minsy
        ET.SubElement(bb,"xmax").text="%s" %x_maxsy
        ET.SubElement(bb,"ymax").text="%s" %y_maxsy


    filename = "%s/%s.xml" %(dirName,name)
    tree = ET.ElementTree(root)
    #tree.write("%s/%s.xml" %(dirName,name),pretty_print=True)
    tree.write("%s" %filename, pretty_print=True)

    return filename #file a single file


In [ ]:
camera_names =['CAM_FRONT', 'CAM_FRONT_RIGHT', 'CAM_FRONT_LEFT', 'CAM_BACK', 'CAM_BACK_RIGHT', 'CAM_BACK_LEFT']

i = 0
detected_items =[]
orig_detected_items=[]
obs = []

file=[]

for camera in camera_names: #iterate through all cameras
    print(camera)
    create_annotation_directory(camera)
    current_dir =os.getcwd()
    dirName ="%s/annotation/%s_anno" %(current_dir,camera) #current directory's name
    #we are looking at one camera now
    for sample_number in range(total_no_of_samples):#look at a single image
        #print(sample_number)
        #get in for a single image




        x_min, x_max,y_min,y_max,width,height, path, boxes, intrinsic_matrix,objects_detected,orig_objects_detected = extract_bounding_box(sample_number, '%s' %camera)
        write_xml_annotation(x_min,x_max,y_min,y_max,width,height,path,boxes,objects_detected)





CAM_FRONT
Directory  /content/annotation/CAM_FRONT_anno  Created 


KeyError: ignored

In [ ]:
print(len(obs))
print(len(orig_detected_items))

print(len(file))

unique = list(set(file))
print(len(unique))

print('total number of files')
3962*6

In [ ]:
print(len(file))
#print(len(unique))

for i in range(len(file)):
    check = file[i]

    for j in range(len(file)):
        if j !=i :
            if check == file[j]:
                print(i)
                print(j)
                print('katie')

In [ ]:
import os.path
from os import listdir
import xml.etree.ElementTree as ET
camera_names =['CAM_FRONT', 'CAM_FRONT_RIGHT', 'CAM_FRONT_LEFT', 'CAM_BACK', 'CAM_BACK_RIGHT', 'CAM_BACK_LEFT']

def list_of_files(camera):
    current_dir =os.getcwd()
    #current_dir ="%s/annotation" %pwd

    dirName ="%s/annotation/%s_anno" %(current_dir,camera)
    files = os.listdir(dirName)

    return files, dirName, current_dir

In [ ]:
total_objects_detected =[]
for camera in camera_names:
    files, dirName,current_dir = list_of_files(camera)
    print(dirName)

    for f in files:
        name_of_file = '%s/%s' %(dirName, f)
        #print(name_of_file)
        w,h,od = extract_data(name_of_file,dirName)
        total_objects_detected = total_objects_detected + od
        #print(od)
        #print(od)


In [ ]:
print(len(total_objects_detected))
print(len(detected_items))

In [ ]:
print(orig_detected_items.count('car'))
print(orig_detected_items.count('pedestrian'))
print(orig_detected_items.count('cyclist'))


add = orig_detected_items.count('car') + orig_detected_items.count('pedestrian') + orig_detected_items.count('cyclist')
print(add)

In [ ]:
import os
print(os.getcwd())

In [ ]:
#print(len(detected_items))
print('Total number of car annotations:')
print(detected_items.count('car'))
print('Total number of pedestrian annotations')
print(detected_items.count('pedestrian'))
print('Total number of cyclist annotations')
print(detected_items.count('cyclist'))

add = detected_items.count('car') + detected_items.count('pedestrian') + detected_items.count('cyclist')
print(add)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
import numpy as np

im = np.array(Image.open('/Volumes/Luthor/nutonomy/nuscenes-devkit-master/python-sdk/%s' %path), dtype=np.uint8)

fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(1, 1, 1)
# Create figure and axes


# Display the image
ax.imshow(im)
#

#print(x)
#print(y)
#print(width)
#print(height)
#print(center_point[0])
#print(center_point[1])

#width = max_x-min_x
#height = max_y-min_y
#for i in range(len(perspective_x)):
#ax.plot(center_point[0], center_point[1], marker ='o', color='b', markersize =30)
#ax.plot(perspective_x[i], perspective_y[i], marker ='o', color='b', markersize =10)
#ax.plot(r2c2[0], r2c2[1], marker ='o', color='b', markersize =10)
#ax.plot(min_x, min_y, marker ='o', color='b', markersize =10)
#ax.plot(max_x, max_y, marker ='o', color='b', markersize =10)

for i in range(len(x_min)):
    if objects_detected[i] =='pedestrian':
        col = 'red'
    elif objects_detected[i] =='car':
        col ='green'
    else:
        col= 'blue'
    rect = patches.Rectangle((x_min[i],y_min[i]),width[i],height[i],linewidth=2,edgecolor='%s' %col,facecolor='none')
#ax.plot(k3[0], k3[1], marker ='o', color='b', markersize =10)
#ax.plot(k4[0], k4[1], marker ='o', color='b', markersize =10)
#ax.plot(k5[0], k5[1], marker ='o', color='b', markersize =10)
#ax.plot(k6[0], k6[1], marker ='o', color='b', markersize =10)
#ax.plot(k7[0], k7[1], marker ='o', color='b', markersize =10)
#ax.plot(k8[0], k8[1], marker ='o', color='b', markersize =10)

#rect = patches.Rectangle((x,y),width,height,linewidth=1,edgecolor='blue',facecolor='none')

# Add the patch to the Axes
    ax.add_patch(rect)
plt.savefig('foo.jpeg')
plt.show()

In [ ]:
#3d render with original bounding boxes
#343
#369

#383
#357
sample_number =348
camera = 'CAM_FRONT_RIGHT'
my_sample = nusc.sample[sample_number]
nusc.render_sample_data(my_sample['data']['%s' %camera])
print(my_sample)
print('this is the path')

nusc.get('sample_data', 'bde261e2ea904fcd86cef6e007bdfdb4')


f1= 'samples/CAM_FRONT_RIGHT/n008-2018-05-21-11-06-59-0400__CAM_FRONT_RIGHT__1526915624869956.jpg'
f2 ='samples/CAM_FRONT_RIGHT/n008-2018-05-21-11-06-59-0400__CAM_FRONT_RIGHT__1526915624869956.jpg'

if f1 ==f2:
    print('katie')

In [ ]:
sample_number =374
camera = 'CAM_FRONT_RIGHT'
my_sample = nusc.sample[sample_number]
nusc.render_sample_data(my_sample['data']['%s' %camera])
print(my_sample)

#'5eedbe17cf2f44e2829567eeeb12f569'

print('this is the path')

nusc.get('sample_data', '2cab2f94315e47eea4e4409d7906db6b')



In [ ]:
#import xml.etree.cElementTree as ET
from lxml import etree as ET
root = ET.Element("annotation")


ET.SubElement(root, "folder").text = "captures_vlc"
ET.SubElement(root, "filename").text = "katie.jpg"
ET.SubElement(root, "path").text = "katie.jpg"

source = ET.SubElement(root, "source")
ET.SubElement(source, "database").text = "nuTonomy-nuscenes"

size = ET.SubElement(root, "size")
ET.SubElement(size, "width").text="Katie"
ET.SubElement(size,"height").text="Kates"
ET.SubElement(size,"depth").text="KM"
ET.SubElement(root, "segmented").text = "0"

ob= ET.SubElement(root, "object")
ET.SubElement(ob,"name").text="ball"
ET.SubElement(ob,"pose").text="Unspecified"
ET.SubElement(ob, "truncated").text="truncated"
ET.SubElement(ob, "difficult").text="0"

bb = ET.SubElement(ob,"bndbox")
ET.SubElement(bb,"xmin").text="xmin"
ET.SubElement(bb,"ymin").text="ymin"
ET.SubElement(bb,"xmax").text="xmax"
ET.SubElement(bb,"ymax").text="ymax"


tree = ET.ElementTree(root)
tree.write("%s.xml" %name,pretty_print=True)

In [ ]:
i = 0
with open('images_with_no_annotations.txt') as f:
    for line in f:
        #print(line)

        i = i +1

In [ ]:
print(i)